In [1]:
import numpy as np 
import pandas as pd 
from scipy.stats import t, norm, poisson, f, chi2
from math import *
import random
import matplotlib.pyplot as plt 

# 功能函数

In [2]:
def C(m,n):
    return factorial(m)/ factorial(n) / factorial(m-n)

def A(m,n):
    return factorial(m) / factorial(m-n)

def arrivalGenerator(rate):
    '''到达时间生成器'''
    t = 0
    while True:
        t += random.expovariate(rate)
        yield t
        
def arrivalGenerator(Lambda, num_customer ,time_precision_digit = 3):
    '''
    到达时间生成器
    param Lambda: 每单位时间到达的客户数
    param num_customer: 客户数量
    param time_precision_digit: 时间单位保留的小数数字长度
    '''
    exp_times = [random.expovariate(Lambda) for i in range(num_customer)] # 顾客的到达时间间隔
    arrival_times = np.round(np.cumsum(exp_times),time_precision_digit) # 顾客的到达时间
    
    # 防止有多个顾客在同一时刻到达
    time_step = 10**(-time_precision_digit)
    copy_arrival_times = []
    for arrival_time in arrival_times:
        if arrival_time not in copy_arrival_times:
            copy_arrival_times.append(arrival_time)
        else:
            copy_arrival_times.append(arrival_time + time_step)
    arrival_times = np.array(copy_arrival_times)        
    return arrival_times

## 指数分布
* 密度函数： $f(x) = \lambda e^{-\lambda x } , while x > 0 $
* 分布（密度累积）函数：$P(X<x)=F(x) = 1 - e^{-\lambda x }$ 

$E(X) = \frac{1}{\lambda}$

$E(X^2) = \frac{2}{\lambda^2}$

$Var(X) = E(X^2) - [E(X)]^2 = \frac{1}{\lambda^2}$

## poisson分布密度概率
$P(X=k)= \frac{\lambda^k}{k!}e^{-\lambda}$

$E(X)=Var(\lambda) = \lambda$

# M /M /1 等待制排队模型
## 仿真

In [3]:
num_customer = 500 # 顾客数量
Lambda = 1/4 # 每单位时间到达的客户数
mu = 1/3 # 每单位时间服务的人数

s = 1 # 服务台的数量
rho = Lambda / mu # 单个服务台的服务强度
rho_s = rho / s # s个服务台的服务强度

exp_times = [random.expovariate(Lambda) for i in range(num_customer)] # 顾客的到达时间间隔
arrival_times = np.cumsum(exp_times) # 顾客的到达时间




In [4]:
class Customer:
    def __init__(self, arrival_time):
        self.time_precision_digit = time_precision_digit
        self.arrival_time = arrival_time # 到达的时刻
        self.wait_time = 0 # 排队等待时间长
        self.stay_time = 0 # 逗留时长
        self.served_time = None # 接受服务的时长
        self.left_time = None # 离开的时刻
    
    def get_info(self):
        self.stay_time = self.left_time - self.arrival_time
        self.wait_time = self.stay_time - self.served_time
        info = [self.arrival_time, self.wait_time, self.served_time, self.left_time, self.stay_time]
        return info
        
class Server:
    def __init__(self, mu):
        self.mu = mu # 单位时间内服务的人数（每分钟服务的人数）
        self.state = 'free' # 柜台的状态：{'bust', 'free'}
        self.served_time_records = [] # 每次服务的服务时长
        self.next_free_time = 0
        self.time_states = [] # 每时刻柜台状态记录
        self.time_service_objects = [] # 每时刻柜台服务的客户ID
    
    def get_service_time(self):
        scale = 1 / self.mu
        return np.random.exponential(scale = scale)
    
    def update_state(self, local_time):
        if local_time < self.next_free_time :
            self.state = 'busy'
        else:
            self.state = 'free'
    
    
class System:
    def __init__(self, arrival_times, server ):
        self.arrival_times = arrival_times # 所有顾客的到达时刻
        self.server = server # 柜台
        self.queue_list = [] # 队列
        self.customer_list = [] # 顾客记录
        self.time_queue_length = [] # 每时刻系统排队服务人数
        self.time_customer_num = [] # 每时刻系统总顾客数
            
    def is_costomer_arrival(self, local_time, error =  1e-4):
        if min(np.abs(local_time - self.arrival_times)) < error:
            return True
        else:
            return False


In [5]:
time_precision_digit = 3 # 时间单位保留的小数数字长度，3表示以1e-3倍时间单位进行迭代更新
time_step = 10**(-time_precision_digit)

num_customer = 1000 # 顾客数量
Lambda = 4 # 每单位时间到达的客户数
mu = 10 # 每单位时间服务的人数

s = 1 # 服务台的数量
rho = Lambda / mu # 单个服务台的服务强度
rho_s = rho / s # s个服务台的服务强度

# 客户的到达时刻
arrival_times = arrivalGenerator(Lambda, num_customer, time_precision_digit )


In [6]:
server = Server(mu)
system = System(arrival_times, server)

time = 0 
time_step = 10**(-time_precision_digit)
max_time = max(arrival_times) * 10
while time < max_time:
    time += time_step
    time = np.round(time, time_precision_digit)
    server.update_state(time)
    if system.is_costomer_arrival(time): # 有客户到
        customer = Customer(time)
        system.queue_list.append(customer)
        
    if len(system.queue_list) > 0 :
        if server.state == 'free':
            server.state = 'busy'
            service_time = round(server.get_service_time(),time_precision_digit)
            server.next_free_time = time + service_time 
            server.served_time_records.append(service_time)
            
            system.queue_list[0].served_time = service_time
            system.queue_list[0].left_time = time + service_time
            system.customer_list.append(system.queue_list[0])
            system.queue_list.remove(system.queue_list[0])
            
    
    if server.state == 'busy':
        server.time_service_objects.append(len(system.customer_list))
        system_customer_num = len(system.queue_list) + 1
    else:
        server.time_service_objects.append(None)
        system_customer_num = len(system.queue_list)
        
    server.time_states.append(server.state)
    system.time_queue_length.append(len(system.queue_list))
    system.time_customer_num.append(system_customer_num)
    
    if len(system.customer_list) == num_customer and time >= system.customer_list[-1].left_time:
        break


In [7]:

system_df = pd.DataFrame({'time': np.arange(0 + time_step ,time + time_step, time_step),
                    '柜台状态': server.time_states,
                   '服务对象ID':server.time_service_objects,
                  '系统排队服务人数': system.time_queue_length,
                 '系统顾客数量': system.time_customer_num })
system_df['柜台是否繁忙'] = system_df['柜台状态'].map({'busy':1, 'free': 0})
system_df.to_csv('system.csv', index = None, encoding='gbk')

In [8]:
datas = []
for customer in system.customer_list:
    item = customer.get_info()
    datas.append(item)
    
customer_df = pd.DataFrame(datas, columns = ['到达时刻','排队等待时长', '服务时长', '离开时刻', '逗留时长'])
customer_df.to_csv('customer.csv', index = None, encoding='gbk')

In [10]:
# 平均队长(平均系统顾客数量)
Ls = system_df['系统顾客数量'].mean()

# 平均排队长（系统排队服务人数）
Lq = system_df['系统排队服务人数'].mean()

# 平均逗留时间
Ws = customer_df['逗留时长'].mean()

# 平均等待时间
Wq = customer_df['排队等待时长'].mean()

# 空闲概率
free_proba = 1 - system_df['柜台是否繁忙'].mean()

# 繁忙概率
busy_proba = system_df['柜台是否繁忙'].mean()

# 顾客到达时必须排队等待的概率
wait_proba = (customer_df['排队等待时长'] > 1e-5).mean()

print('平均队长:', round(Ls,3))
print('平均排队长:', round(Lq,3))
print('平均逗留时间:', round(Ws,3))
print('平均等待时间:', round(Wq,3))
print('柜员繁忙概率:', round(busy_proba,3))
print('顾客到达时必须排队等待的概率:', round(wait_proba,3))


平均队长: 0.654
平均排队长: 0.255
平均逗留时间: 0.162
平均等待时间: 0.063
柜员繁忙概率: 0.399
顾客到达时必须排队等待的概率: 0.392


## 理论计算

In [11]:
# 平均队长(平均系统顾客数量)
Ls = Lambda / (mu - Lambda)

# 平均排队长（系统排队服务人数）
Lq = Lambda**2 / (mu * (mu - Lambda))

# 平均逗留时间
Ws = 1 / (mu - Lambda)

# 平均等待时间
Wq = Lambda / (mu * (mu - Lambda))

# 平均闲期
mean_free = 1 / Lambda

# 平均忙期
maen_busy = 1 / (mu - Lambda)

# 空闲概率
free_proba = 1 - Lambda / mu

# 繁忙概率
busy_proba = Lambda / mu

print('平均队长:', round(Ls,3))
print('平均排队长:', round(Lq,3))
print('平均逗留时间:', round(Ws,3))
print('平均等待时间:', round(Wq,3))
# print('柜员平均闲期:', round(mean_free,3))
# print('柜员平均忙期:', round(maen_busy,3))
print('柜员繁忙概率:', round(busy_proba,3))


平均队长: 0.667
平均排队长: 0.267
平均逗留时间: 0.167
平均等待时间: 0.067
柜员繁忙概率: 0.4


# M /M /s 等待制排队模型
## 仿真

In [12]:
class Customer:
    def __init__(self, arrival_time):
        self.time_precision_digit = time_precision_digit
        self.arrival_time = arrival_time # 到达的时刻
        self.wait_time = 0 # 排队等待时间长
        self.stay_time = 0 # 逗留时长
        self.served_time = None # 接受服务的时长
        self.left_time = None # 离开的时刻
    
    def get_info(self):
        self.stay_time = self.left_time - self.arrival_time
        self.wait_time = self.stay_time - self.served_time
        info = [self.arrival_time, self.wait_time, self.served_time, self.left_time, self.stay_time]
        return info
        
class Server:
    def __init__(self, mu):
        self.mu = mu # 单位时间内服务的人数（每分钟服务的人数）
        self.state = 'free' # 柜台的状态：{'bust', 'free'}
        self.served_time_records = [] # 每次服务的服务时长
        self.next_free_time = 0
        self.time_states = [] # 每时刻柜台状态记录
        self.service_object = None
        self.time_service_objects = [] # 每时刻柜台服务的客户ID
    
    def get_service_time(self):
        scale = 1 / self.mu
        return np.random.exponential(scale = scale)
    
    def update_state(self, local_time):
        if local_time < self.next_free_time :
            self.state = 'busy'
        else:
            self.state = 'free'
    
    
class System:
    def __init__(self, arrival_times, server_list ):
        self.num_server = len(server_list)
        self.arrival_times = arrival_times # 所有顾客的到达时刻
        self.server_list = server_list # 所有柜台组成的list
        self.queue_list = [] # 队列
        self.customer_list = [] # 顾客记录
        self.time_queue_length = [] # 每时刻系统排队服务人数
        self.time_customer_num = [] # 每时刻系统总顾客数
            
    def is_costomer_arrival(self, local_time, error =  1e-4):
        '''判断顾客在当前时刻是否到达'''
        if min(np.abs(local_time - self.arrival_times)) < error:
            return True
        else:
            return False
    
    def random_select_free_server(self):
        '''随机选择一个空闲状态的柜员'''
        free_inds = [] 
        for i, server in enumerate(self.server_list):
            if server.state == 'free':
                free_inds.append(i)
        if len(free_inds) > 0 :
            free_ind = np.random.choice(free_inds)
        else:
            free_ind = None
        return free_ind
    
    def get_system_customer_num(self):
        '''获取系统中顾客的数量（包括正在排队的）'''
        num = 0  # 正在接待服务的顾客数量
        for i, server in enumerate(self.server_list):
            if server.state == 'busy':
                num += 1 
                
        total = num + len(self.queue_list)
        return total
    

In [13]:
def arrivalGenerator(Lambda, num_customer ,time_precision_digit = 3):
    '''
    到达时间生成器
    param Lambda: 每单位时间到达的客户数
    param num_customer: 客户数量
    param time_precision_digit: 时间单位保留的小数数字长度
    '''
    exp_times = [random.expovariate(Lambda) for i in range(num_customer)] # 顾客的到达时间间隔
    arrival_times = np.round(np.cumsum(exp_times),time_precision_digit) # 顾客的到达时间
    
    # 防止有多个顾客在同一时刻到达
    time_step = 10**(-time_precision_digit)
    copy_arrival_times = []
    for arrival_time in arrival_times:
        if arrival_time not in copy_arrival_times:
            copy_arrival_times.append(arrival_time)
        else:
            copy_arrival_times.append(arrival_time + time_step)
    arrival_times = np.array(copy_arrival_times)        
    return arrival_times

In [14]:
time_precision_digit = 3 # 时间单位保留的小数数字长度，3表示以1e-3倍时间单位进行迭代更新
time_step = 10**(-time_precision_digit)

num_customer = 1000 # 顾客数量
Lambda = 0.9 # 每单位时间到达的客户数

mu_list = [0.4, 0.4, 0.4] # 各个柜台每单位时间服务的人数
s = len(mu_list) # 柜台的数量


# 客户的到达时刻
arrival_times = arrivalGenerator(Lambda, num_customer, time_precision_digit )


In [15]:
server_list = [Server(mu) for mu in mu_list]
system = System(arrival_times, server_list)

time = 0 
max_time = max(arrival_times) * 10
while time < max_time:
    time += time_step
    time = np.round(time, time_precision_digit)
    
    for server in system.server_list:
        server.update_state(time)
        
    if system.is_costomer_arrival(time): # 有客户到
        customer = Customer(time)
        system.queue_list.append(customer)
        
    if len(system.queue_list) > 0 :
        free_ind = system.random_select_free_server()
        if free_ind is not None:
            server = system.server_list[free_ind]
            server.state = 'busy'
            service_time = round(server.get_service_time(),time_precision_digit)
            server.next_free_time = time + service_time 
            server.served_time_records.append(service_time)
            
            system.queue_list[0].served_time = service_time
            system.queue_list[0].left_time = time + service_time
            system.customer_list.append(system.queue_list[0])
            system.queue_list.remove(system.queue_list[0])
            service_object = len(system.customer_list)
            server.service_object = service_object
    
    
    for server in system.server_list:
        if server.state == 'busy':
            server.time_service_objects.append(server.service_object)
        else:
            server.time_service_objects.append(None)
            
        server.time_states.append(server.state)
    
    system_customer_num = system.get_system_customer_num()
    system.time_customer_num.append(system_customer_num)
    system.time_queue_length.append(len(system.queue_list))
    if len(system.customer_list) == num_customer and time >= system.customer_list[-1].left_time:
        break

In [16]:
data_dict = {'time': np.arange(0 + time_step ,time + time_step, time_step),
                  '系统排队服务人数': system.time_queue_length,
                 '系统顾客数量': system.time_customer_num }

for i, server in enumerate(system.server_list):
    key = '柜台{}状态'.format(i+1)
    data_dict[key] = server.time_states
    
    key = "柜台{}服务对象ID".format(i+1)
    data_dict[key] = server.time_service_objects

system_df = pd.DataFrame(data_dict)
system_df['系统中繁忙柜台的数量'] = 0 

for i, server in enumerate(system.server_list):
    key = '柜台{}状态'.format(i+1)
    col = "柜台{}是否繁忙".format(i+1)
    system_df[col] = system_df[key].map({'busy':1, 'free': 0})
    system_df['系统中繁忙柜台的数量'] += system_df[col]

system_df.to_csv('system.csv', index = None, encoding='gbk')

In [17]:
datas = []
for customer in system.customer_list:
    item = customer.get_info()
    datas.append(item)
    
customer_df = pd.DataFrame(datas, columns = ['到达时刻','排队等待时长', '服务时长', '离开时刻', '逗留时长'])
customer_df.to_csv('customer.csv', index = None, encoding='gbk')

In [18]:
# 平均队长(平均系统顾客数量)
Ls = system_df['系统顾客数量'].mean()

# 平均排队长（系统排队服务人数）
Lq = system_df['系统排队服务人数'].mean()

# 平均逗留时间
Ws = customer_df['逗留时长'].mean()

# 平均等待时间
Wq = customer_df['排队等待时长'].mean()

# 顾客到达时必须排队等待的概率
wait_proba = (customer_df['排队等待时长'] > 1e-5).mean()

# 整个系统空闲的概率
system_free_proba = (system_df['系统中繁忙柜台的数量'] == 0).mean()

print('平均队长:', round(Ls,3))
print('平均排队长:', round(Lq,3))
print('平均逗留时间:', round(Ws,3))
print('平均等待时间:', round(Wq,3))
print('顾客到达时必须排队等待的概率:', round(wait_proba,3))

for i in range(s):
    col = '柜台{}是否繁忙'.format(i+1)
    busy_proba = system_df[col].mean()     # 繁忙概率
    print('柜台{}繁忙概率:{:.3f}'.format(i+1, busy_proba))

print('整个系统空闲的概率:', round(system_free_proba,5))



平均队长: 2.963
平均排队长: 0.899
平均逗留时间: 3.364
平均等待时间: 1.02
顾客到达时必须排队等待的概率: 0.463
柜台1繁忙概率:0.696
柜台2繁忙概率:0.692
柜台3繁忙概率:0.676
整个系统空闲的概率: 0.09392


## 理论计算

In [19]:
mu = 0.4
rho = Lambda / mu
def cal_system_free_proba(rho, s):
    '''整个系统空闲的概率的概率'''
    # 系统中0个顾客的概率
    p0 = 0
    for i in range(s):
        p0 += rho**i / A(i,i) 
    p0 += rho**s / (A(s,s) * (1 - rho / s))
    p0 = 1 / p0
    return p0

def cal_wait_proba(rho, s):
    '''顾客到达系统时需要等待的概率'''
    p0 = cal_system_free_proba(rho, s)
    proba = rho**s / A(s,s) / (1 - rho / s ) * p0
    return proba

# 整个系统空闲的概率
system_free_proba = cal_system_free_proba(rho, s)

# 顾客到达系统时需要等待的概率
wait_proba = cal_wait_proba(rho, s)

# 平均排队长
Lq = wait_proba * rho / s / (1 - rho / s)

# 平均队长(包括正在服务的顾客)
Ls = Lq + rho

# 平均逗留时间
Ws = Ls / Lambda

# 平均等待时间
Wq = Lq / Lambda


print('平均队长:', round(Ls,3))
print('平均排队长:', round(Lq,3))
print('平均逗留时间:', round(Ws,3))
print('平均等待时间:', round(Wq,3))
print('顾客到达时必须排队等待的概率:', round(wait_proba,3))
print('整个系统空闲的概率:', round(system_free_proba,5))


平均队长: 3.953
平均排队长: 1.703
平均逗留时间: 4.393
平均等待时间: 1.893
顾客到达时必须排队等待的概率: 0.568
整个系统空闲的概率: 0.07477


# M/M/s/K
顾客的相继到达时间服从参数为 λ 的负指 数分布，服务台个数为 s ，每个服务台服务时间相互独立，且服从参数为 μ 的负指数分
布，系统的空间为 K 。
## 仿真

In [20]:
class Customer:
    def __init__(self, arrival_time):
        self.time_precision_digit = time_precision_digit
        self.arrival_time = arrival_time # 到达的时刻
        self.wait_time = 0 # 排队等待时间长
        self.stay_time = 0 # 逗留时长
        self.served_time = None # 接受服务的时长
        self.left_time = None # 离开的时刻
    
    def get_info(self):
        self.stay_time = self.left_time - self.arrival_time
        self.wait_time = self.stay_time - self.served_time
        info = [self.arrival_time, self.wait_time, self.served_time, self.left_time, self.stay_time]
        return info
        
class Server:
    def __init__(self, mu):
        self.mu = mu # 单位时间内服务的人数（每分钟服务的人数）
        self.state = 'free' # 柜台的状态：{'bust', 'free'}
        self.served_time_records = [] # 每次服务的服务时长
        self.next_free_time = 0
        self.time_states = [] # 每时刻柜台状态记录
        self.service_object = None
        self.time_service_objects = [] # 每时刻柜台服务的客户ID
    
    def get_service_time(self):
        scale = 1 / self.mu
        return np.random.exponential(scale = scale)
    
    def update_state(self, local_time):
        if local_time < self.next_free_time :
            self.state = 'busy'
        else:
            self.state = 'free'
    
    
class System:
    def __init__(self, arrival_times, server_list, K ):
        self.num_server = len(server_list)
        self.arrival_times = arrival_times # 所有顾客的到达时刻
        self.server_list = server_list # 所有柜台组成的list
        self.K = K # 系统空间的上线（系统中顾客的最大数量，包括正在接受服务的顾客）
        self.queue_list = [] # 队列
        self.customer_list = [] # 顾客记录
        self.time_queue_length = [] # 每时刻系统排队服务人数
        self.time_customer_num = [] # 每时刻系统总顾客数
            
    def is_costomer_arrival(self, local_time, error =  1e-4):
        '''判断顾客在当前时刻是否到达'''
        if min(np.abs(local_time - self.arrival_times)) < error:
            return True
        else:
            return False
    
    def random_select_free_server(self):
        '''随机选择一个空闲状态的柜员'''
        free_inds = [] 
        for i, server in enumerate(self.server_list):
            if server.state == 'free':
                free_inds.append(i)
        if len(free_inds) > 0 :
            free_ind = np.random.choice(free_inds)
        else:
            free_ind = None
        return free_ind
    
    def get_system_customer_num(self):
        '''获取系统中顾客的数量（包括正在排队的）'''
        num = 0  # 正在接待服务的顾客数量
        for i, server in enumerate(self.server_list):
            if server.state == 'busy':
                num += 1 
                
        total = num + len(self.queue_list)
        return total
    

In [21]:
def arrivalGenerator(Lambda, num_customer ,time_precision_digit = 3):
    '''
    到达时间生成器
    param Lambda: 每单位时间到达的客户数
    param num_customer: 客户数量
    param time_precision_digit: 时间单位保留的小数数字长度
    '''
    exp_times = [random.expovariate(Lambda) for i in range(num_customer)] # 顾客的到达时间间隔
    arrival_times = np.round(np.cumsum(exp_times),time_precision_digit) # 顾客的到达时间
    
    # 防止有多个顾客在同一时刻到达
    time_step = 10**(-time_precision_digit)
    copy_arrival_times = []
    for arrival_time in arrival_times:
        if arrival_time not in copy_arrival_times:
            copy_arrival_times.append(arrival_time)
        else:
            copy_arrival_times.append(arrival_time + time_step)
    arrival_times = np.array(copy_arrival_times)        
    return arrival_times

In [22]:
time_precision_digit = 3 # 时间单位保留的小数数字长度，3表示以1e-3倍时间单位进行迭代更新
time_step = 10**(-time_precision_digit)

num_customer = 2000 # 顾客数量
Lambda = 2 # 每单位时间到达的客户数

mu_list = [1/2,1/2] # 各个柜台每单位时间服务的人数
s = len(mu_list) # 柜台的数量

K = 5 # 系统中顾客的最大数量，包括正在接受服务的顾客

# 客户的到达时刻
arrival_times = arrivalGenerator(Lambda, num_customer, time_precision_digit )


In [23]:
server_list = [Server(mu) for mu in mu_list]
system = System(arrival_times, server_list, K )

time = 0 
max_time = max(arrival_times) * 10 # 迭代的最大时间
max_left_time = 0 # 所有顾客中离开时间的最大值
while time < max_time:
    time += time_step
    time = np.round(time, time_precision_digit)
    
    for server in system.server_list:
        server.update_state(time)
        
    if system.is_costomer_arrival(time): # 有客户到
        customer = Customer(time)
        system.queue_list.append(customer)
        
    system_customer_num = system.get_system_customer_num() # 统计系统顾客数量（包括在排队中的）
    if len(system.queue_list) > 0 and system_customer_num <= system.K:
        free_ind = system.random_select_free_server()
        if free_ind is not None:
            server = system.server_list[free_ind]
            server.state = 'busy'
            service_time = round(server.get_service_time(),time_precision_digit)
            server.next_free_time = time + service_time 
            server.served_time_records.append(service_time)
            system.queue_list[0].served_time = service_time
            system.queue_list[0].left_time = time + service_time
            
            if system.queue_list[0].left_time > max_left_time:
                max_left_time = system.queue_list[0].left_time
                
            system.customer_list.append(system.queue_list[0])
            system.queue_list.remove(system.queue_list[0])
            service_object = len(system.customer_list)
            server.service_object = service_object
            
                
    elif system_customer_num > system.K:
        system.queue_list[-1].served_time = 0
        system.queue_list[-1].left_time = time 
        system.customer_list.append(system.queue_list[-1])
        system.queue_list.remove(system.queue_list[-1])
    
    for server in system.server_list:
        if server.state == 'busy':
            server.time_service_objects.append(server.service_object)
        else:
            server.time_service_objects.append(None)
            
        server.time_states.append(server.state)
    
    system_customer_num = system.get_system_customer_num()
    system.time_customer_num.append(system_customer_num)
    system.time_queue_length.append(len(system.queue_list))
    if len(system.customer_list) == num_customer and time >= max_left_time:
        break

In [24]:
data_dict = {'time': np.arange(0 + time_step ,time + time_step, time_step),
                  '系统排队服务人数': system.time_queue_length,
                 '系统顾客数量': system.time_customer_num }

for i, server in enumerate(system.server_list):
    key = '柜台{}状态'.format(i+1)
    data_dict[key] = server.time_states
    
    key = "柜台{}服务对象ID".format(i+1)
    data_dict[key] = server.time_service_objects

system_df = pd.DataFrame(data_dict)
system_df['系统中繁忙柜台的数量'] = 0 

for i, server in enumerate(system.server_list):
    key = '柜台{}状态'.format(i+1)
    col = "柜台{}是否繁忙".format(i+1)
    system_df[col] = system_df[key].map({'busy':1, 'free': 0})
    system_df['系统中繁忙柜台的数量'] += system_df[col]

system_df.to_csv('system.csv', index = None, encoding='gbk')

In [25]:
datas = []
for customer in system.customer_list:
    item = customer.get_info()
    datas.append(item)
    
customer_df = pd.DataFrame(datas, columns = ['到达时刻','排队等待时长', '服务时长', '离开时刻', '逗留时长'])
customer_df.to_csv('customer.csv', index = None, encoding='gbk')

In [26]:
# 平均队长(平均系统顾客数量)
Ls = system_df['系统顾客数量'].mean()

# 平均排队长（系统排队服务人数）
Lq = system_df['系统排队服务人数'].mean()

# 平均逗留时间(不计算损失的客户)
mask = customer_df['服务时长'] > 0
Ws = customer_df[mask]['逗留时长'].mean()

# 平均等待时间(不计算损失的客户)
Wq = customer_df[mask]['排队等待时长'].mean()

# 顾客到达时必须排队等待的概率(不计算损失的客户)
wait_proba = (customer_df['排队等待时长'] > 1e-5).mean()

# 整个系统空闲的概率
system_free_proba = (system_df['系统中繁忙柜台的数量'] == 0).mean()

# 顾客流失率
lose_ratio = (customer_df['服务时长'] == 0).mean() 

# 被占用的柜台的平均数为
mean_busy_server_num = system_df['系统中繁忙柜台的数量'].mean()

print('平均队长:', round(Ls,3))
print('平均排队长:', round(Lq,3))
print('平均逗留时间(不计算损失的客户):', round(Ws,3))
print('平均等待时间(不计算损失的客户):', round(Wq,3))
print('顾客到达时必须排队等待的概率(不计算损失的客户):', round(wait_proba,3))

for i in range(s):
    col = '柜台{}是否繁忙'.format(i+1)
    busy_proba = system_df[col].mean()     # 繁忙概率
    print('柜台{}繁忙概率:{:.3f}'.format(i+1, busy_proba))

print('整个系统空闲的概率:', round(system_free_proba,5))

print('顾客流失率:', round(lose_ratio,3))

print('被占用的柜台的平均数为:', round(mean_busy_server_num,3))

平均队长: 3.993
平均排队长: 2.066
平均逗留时间(不计算损失的客户): 3.991
平均等待时间(不计算损失的客户): 2.065
顾客到达时必须排队等待的概率(不计算损失的客户): 0.47
柜台1繁忙概率:0.958
柜台2繁忙概率:0.968
整个系统空闲的概率: 0.01407
顾客流失率: 0.475
被占用的柜台的平均数为: 1.926


## 理论计算

In [27]:
# 平均逗留时间(不计算损失的客户)
mask = customer_df['服务时长'] > 0
customer_df[mask]['逗留时长'].mean()

3.990562857142868

In [28]:
def cal_system_free_proba(rho, s, K):
    '''整个系统空闲的概率的概率'''
    # 系统中0个顾客的概率
    total = 0
    for i in range(s):
        total += rho**i / A(i,i) 
        
    if rho / s == 1:
        p0 = total + rho**s / A(s,s) * (K - s + 1)
    else:
        p0 = total + rho**s * (1 - (rho/s)**(K-s+1) ) / (A(s,s) * (1 - rho/s))
    p0 = 1 / p0
    return p0

def cal_system_customer_num_proba(rho, s, K, n):
    '''整个系统中出现n个顾客（包括正在服务）的概率'''
    p0 = cal_system_free_proba(rho, s, K)
    if n < s :
        pn = rho**n / A(n,n) * p0
    elif n >= s and n <= K:
        pn = rho**n / A(s,s) * p0 / s**(n-s)
    else:
        pn = 0
    return pn

def cal_mean_customer_num(rho, s, K):
    '''计算平均队长（系统中顾客人数，包括正在排队的）'''
    total = 0 
    for n in range(K+1):
        total += n * cal_system_customer_num_proba(rho, s, K, n)
    return total


def cal_mean_queue_length(rho, s, K):
    '''计算平均排队长（系统中排队顾客人数）'''
    total = 0 
    for n in range(s,K+1):
        total += (n-s) * cal_system_customer_num_proba(rho, s, K, n)
    return total




In [29]:
mu = 1/2
rho = Lambda / mu

# 平均队长(平均系统顾客数量)
Ls = cal_mean_customer_num(rho, s, K)

# 平均排队长（系统排队服务人数）
Lq = cal_mean_queue_length(rho, s, K)

# 平均逗留时间(不计算损失的客户)
pK = cal_system_customer_num_proba(rho, s, K, K) # 系统满载的概率(顾客损失率)
Ws = Ls / (Lambda * (1 - pK))

# 平均等待时间(不计算损失的客户)
Wq = Ws - 1 / mu

# 整个系统空闲的概率
system_free_proba = cal_system_customer_num_proba(rho, s, K, 0)

# 顾客流失率
lose_ratio = pK = cal_system_customer_num_proba(rho, s, K, K) # 系统满载的概率(顾客损失率)

# 被占用的柜台的平均数为
mean_busy_server_num = Ls - Lq


print('平均队长:', round(Ls,3))
print('平均排队长:', round(Lq,3))
print('平均逗留时间(不计算损失的客户):', round(Ws,3))
print('平均等待时间(不计算损失的客户):', round(Wq,3))

print('整个系统空闲的概率:', round(system_free_proba,5))

print('顾客流失率:', round(lose_ratio,3))

print('被占用的柜台的平均数为:', round(mean_busy_server_num,3))

平均队长: 4.128
平均排队长: 2.176
平均逗留时间(不计算损失的客户): 4.23
平均等待时间(不计算损失的客户): 2.23
整个系统空闲的概率: 0.008
顾客流失率: 0.512
被占用的柜台的平均数为: 1.952
